In [12]:
import pandas as pd

In [13]:
# set fred api key
fred_api_key = "653ed88f4bdd6c12141a8c80288845c4"
cpi_series_id = "CPIAUCSL"

In [14]:
# get cpi data
cpi = pd.read_json(f"https://api.stlouisfed.org/fred/series/observations?series_id={cpi_series_id}&api_key={fred_api_key}&file_type=json")

# make a dataframe from the observations
cpi = pd.DataFrame(cpi["observations"].values.tolist())

# select the date and value columns
cpi = cpi[["date", "value"]]
cpi.columns = ["date", "cpi"]

# convert date to datetime and cpi to numeric
cpi["date"] = pd.to_datetime(cpi["date"])
cpi['cpi'] = pd.to_numeric(cpi['cpi'])

# generate a yoy change column
cpi["cpi_yoy"] = cpi["cpi"].pct_change(12)

cpi

,date,cpi,cpi_yoy
0,1947-01-01,21.480,NaN
1,1947-02-01,21.620,NaN
2,1947-03-01,22.000,NaN
3,1947-04-01,22.000,NaN
4,1947-05-01,21.950,NaN
...,...,...,...
930,2024-07-01,313.534,0.029236
931,2024-08-01,314.121,0.025912
932,2024-09-01,314.686,0.024075
933,2024-10-01,315.454,0.025763


In [15]:
# get zori data
zori = pd.read_csv("data/zori_metro_long_clean.csv")

# select only us data
zori = zori[zori["name"] == "United States"]

# restrict to cols of interest
zori = zori[["date", "zori"]]

# convert date to datetime
zori["date"] = pd.to_datetime(zori["date"])

zori

,date,zori
0,2015-02-01,1204.835243
628,2015-03-01,1211.392934
1256,2015-04-01,1220.361278
1884,2015-05-01,1229.453789
2512,2015-06-01,1238.715977
...,...,...
71592,2024-08-01,1984.255546
72220,2024-09-01,1988.302243
72848,2024-10-01,1989.313398
73476,2024-11-01,1985.557293


In [16]:
zori.dtypes

date    datetime64[ns]
zori           float64
dtype: object

In [17]:
# date is currently the last day of the month, so we need to shift it to the first day of the month
zori["date"] = zori["date"] + pd.offsets.MonthBegin(-1)
zori

,date,zori
0,2015-01-01,1204.835243
628,2015-02-01,1211.392934
1256,2015-03-01,1220.361278
1884,2015-04-01,1229.453789
2512,2015-05-01,1238.715977
...,...,...
71592,2024-07-01,1984.255546
72220,2024-08-01,1988.302243
72848,2024-09-01,1989.313398
73476,2024-10-01,1985.557293


In [18]:
# add yoy change column
zori["zori_yoy"] = zori["zori"].pct_change(12)
zori

,date,zori,zori_yoy
0,2015-01-01,1204.835243,NaN
628,2015-02-01,1211.392934,NaN
1256,2015-03-01,1220.361278,NaN
1884,2015-04-01,1229.453789,NaN
2512,2015-05-01,1238.715977,NaN
...,...,...,...
71592,2024-07-01,1984.255546,0.034275
72220,2024-08-01,1988.302243,0.033474
72848,2024-09-01,1989.313398,0.033217
73476,2024-10-01,1985.557293,0.032856


In [19]:
# merge the two dataframes
# we want to keep all the dates in the zori dataframe
# so we use a left join
rent_inflation = pd.merge(
    zori,
    cpi,
    on="date",
    how="left"
    )

rent_inflation = rent_inflation[["date", "zori_yoy", "cpi_yoy"]]

rent_inflation

,date,zori_yoy,cpi_yoy
0,2015-01-01,NaN,-0.002299
1,2015-02-01,NaN,-0.000870
2,2015-03-01,NaN,-0.000220
3,2015-04-01,NaN,-0.001040
4,2015-05-01,NaN,0.000350
...,...,...,...
114,2024-07-01,0.034275,0.029236
115,2024-08-01,0.033474,0.025912
116,2024-09-01,0.033217,0.024075
117,2024-10-01,0.032856,0.025763


In [20]:
# melt the dataframe
rent_inflation = rent_inflation.melt(id_vars="date", var_name="metric", value_name="yoy_change")

In [21]:
# change the values of the metric column to zori_yoy:Rent and cpi_yoy:CPI
# make a dictionary to map the old values to the new values
metric_map = {
    "zori_yoy": "Rent",
    "cpi_yoy": "CPI"
}

# use the map method to change the values
rent_inflation["metric"] = rent_inflation["metric"].map(metric_map)

In [22]:
# save to csv
rent_inflation.to_csv("data/rent_inflation.csv", index=False)